
###Во оваа лабораторика вежба фокусот е на time series предвидувања, односно предвидување на Close (BTC) вредноста во даденото податочно множество за Bitcoin. Ваша задача е со помош на decision tree и boosting методи да ја предвидите бараната вредност земајќи ги во предвид релевантните колони од податочното множество, но дополнително и вредноста на Close (BTC) во изминатите четири дена, бидејќи вредноста на Bitcoin-от во даден момент зависи од нејзината вредност во изминатите денови.

Hint : внимавајте како ќе го поделите множеството на train и test бидејќи се работи податоци во временска серија

##Симнете го податочното множество

In [ ]:
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np
from xgboost import XGBRegressor

In [ ]:
!pip install gdown==v4.6.3

In [ ]:
!gdown 14YlgjT_M8kDrvDKEv864fs7ZyI1lMErO

Downloading...
From: https://drive.google.com/uc?id=14YlgjT_M8kDrvDKEv864fs7ZyI1lMErO
To: /content/crypto_data.csv
100% 90.0k/90.0k [00:00<00:00, 113MB/s]


##Направете претпроцесирање на податоците

In [ ]:
df = pd.read_csv('crypto_data.csv')

In [ ]:
df.head()

,Close (BNB),Close (BTC),Close (USDT),Close (ETH),Close(BTC)_prev_1_day,Close(BTC)_prev_2_day,Close(BTC)_prev_3_day,Close(BTC)_prev_4_day
Date,,,,,,,,
01-01-18,8.414610,13657.200200,1.007280,772.640991,NaN,NaN,NaN,NaN
01-01-19,6.075273,3843.520020,1.018335,140.819412,13657.200200,NaN,NaN,NaN
01-01-20,13.689083,7200.174316,0.999836,130.802002,3843.520020,13657.200200,NaN,NaN
01-01-21,37.905010,29374.152340,1.001916,730.367554,7200.174316,3843.520020,13657.20020,NaN
01-01-22,527.352722,47686.812500,1.000500,3769.697021,29374.152340,7200.174316,3843.52002,13657.2002


In [ ]:
df.isnull().sum()

In [ ]:
df = df.set_index('Date')
df = df.sort_index()

In [ ]:
lag = 4
for i in range(1, lag+1):
  df[f'Close(BTC)_prev_{i}_day'] = df["Close (BTC)"].shift(i)

In [ ]:
df = df.dropna(axis=0)

##Поделето го на тренирачко и тестирачко множество во сооднос 80:20

In [ ]:
X = df.copy()
X = X.drop(columns=["Close (BTC)", "Close (BNB)", "Close (USDT)", "Close (ETH)"])

In [ ]:
Y = df["Close (BTC)"]

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, shuffle=False)

##Искористете соодветен Boosting модел за проблемот и тестирајте го моделот на тестирачкото множество

In [ ]:
xgb = XGBRegressor(n_estimators=10, max_depth=5)

In [ ]:
xgb.fit(X_train, Y_train)

In [ ]:
preds = xgb.predict(X_test)

In [ ]:
mean_squared_error(preds, Y_test), mean_absolute_error(preds, Y_test), r2_score(preds, Y_test)

##Искористете соодветен Decision Tree модел за проблемот и тестирајте го моделот на тестирачкото множество

##Споредете ги резултатите добиени од двата модели со соодветни метрики

##Bonus Task: Направете визуелизации кои го илустрираат процесот на предвидување на двата горенавередени модели